In [1]:
import pandas as pd

import numpy as np

In [3]:
annotation_droplet = pd.read_csv('/home/ubuntu/tabula-muris/00_data_ingest/18_global_annotation_csv/annotations_droplet.csv')
print(annotation_droplet.shape)
annotation_droplet.head()

(55656, 20)


/home/ubuntu/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (10,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cell,cell_ontology_class,cell_ontology_id,channel,cluster.ids,free_annotation,mouse.id,mouse.sex,subsetA,subsetA_cluster.ids,subsetB,subsetB_cluster.ids,subsetC,subsetC_cluster.ids,subsetD,subsetD_cluster.ids,subtissue,tissue,tissue_tSNE_1,tissue_tSNE_2
0,10X_P7_8_AAACGGGAGGATATAC,myeloid cell,CL:0000763,10X_P7_8,20,dendritic cells and interstital macrophages,3-F-56,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lung,17.024721,-32.902836
1,10X_P7_8_AAACGGGTCTCGTATT,alveolar macrophage,CL:0000583,10X_P7_8,5,NaN,3-F-56,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lung,25.160619,25.066566
2,10X_P7_8_AAAGATGCAGATCTGT,B cell,CL:0000236,10X_P7_8,12,NaN,3-F-56,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lung,1.740567,46.488878
3,10X_P7_8_AAATGCCAGATAGTCA,natural killer cell,CL:0000623,10X_P7_8,7,NaN,3-F-56,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lung,-31.647934,-2.208061
4,10X_P7_8_AAATGCCCAAACTGCT,T cell,CL:0000084,10X_P7_8,21,NaN,3-F-56,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lung,-37.281266,-5.619565


In [6]:
droplet_channels = annotation_droplet[['channel', 'tissue']].drop_duplicates()
print(droplet_channels.shape)
droplet_channels

(28, 2)


,channel,tissue
0,10X_P7_8,Lung
621,10X_P7_9,Lung
2141,10X_P8_12,Lung
2925,10X_P8_13,Lung
5449,10X_P7_2,Marrow
7181,10X_P7_3,Marrow
9101,10X_P4_0,Tongue
12202,10X_P4_1,Tongue
14690,10X_P7_10,Tongue
16639,10X_P4_7,Spleen


In [7]:
spleen_kidney = 'Spleen', 'Kidney'

droplet_channels.query('tissue in @spleen_kidney')

,channel,tissue
16639,10X_P4_7,Spleen
20078,10X_P7_6,Spleen
45065,10X_P4_5,Kidney
45675,10X_P4_6,Kidney
46582,10X_P7_5,Kidney


In [11]:
! aws s3 ls --human-readable s3://czbiohub-maca/10x_data/10X_P4_7/

2017-08-22 12:46:51  324.6 MiB 10X_P4_7.mus.cell-gene.csv
2017-08-22 12:42:56   21.9 GiB 10X_P4_7.tgz
2017-08-31 21:56:04   64.2 KiB barcodes.tsv
2017-08-31 22:00:17  339.8 KiB genes.tsv
2017-08-31 22:04:32   51.9 MiB matrix.mtx
2017-08-22 12:26:46  616 Bytes metrics_summary.csv
2018-06-19 02:48:08   21.7 GiB possorted_genome_bam.bam
2018-06-19 02:48:08    5.7 MiB possorted_genome_bam.bam.bai
2017-09-15 22:24:33   15.2 MiB raw_gene_bc_matrices_h5.h5
2017-08-22 12:26:45    2.5 MiB web_summary.html


In [18]:
s3_prefix = 's3://czbiohub-maca/10x_data/'

In [36]:
ksizes = ",".join(map(str, (21, 27, 33, 51)))
num_hashes = 1000

In [37]:
output_prefix = f's3://olgabot-maca/10x/sourmash_compute/ksizes={ksizes}_num_hashes={num_hashes}/'
output_prefix

's3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/'

In [39]:
samples = droplet_channels.query('tissue in @spleen_kidney')
samples['ksizes'] = ksizes
samples['num_hashes'] = num_hashes
samples.head()

/home/ubuntu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ubuntu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,channel,tissue,ksizes,num_hashes
16639,10X_P4_7,Spleen,"21,27,33,51",1000
20078,10X_P7_6,Spleen,"21,27,33,51",1000
45065,10X_P4_5,Kidney,"21,27,33,51",1000
45675,10X_P4_6,Kidney,"21,27,33,51",1000
46582,10X_P7_5,Kidney,"21,27,33,51",1000


In [40]:
samples['tenx'] = s3_prefix + samples['channel']
samples.head()

/home/ubuntu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,channel,tissue,ksizes,num_hashes,tenx
16639,10X_P4_7,Spleen,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P4_7
20078,10X_P7_6,Spleen,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P7_6
45065,10X_P4_5,Kidney,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P4_5
45675,10X_P4_6,Kidney,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P4_6
46582,10X_P7_5,Kidney,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P7_5


In [41]:
samples['id'] = samples.apply(lambda x: '{tissue}_{channel}'.format(**x), axis=1)
samples.head()

/home/ubuntu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,channel,tissue,ksizes,num_hashes,tenx,id
16639,10X_P4_7,Spleen,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P4_7,Spleen_10X_P4_7
20078,10X_P7_6,Spleen,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P7_6,Spleen_10X_P7_6
45065,10X_P4_5,Kidney,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P4_5,Kidney_10X_P4_5
45675,10X_P4_6,Kidney,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P4_6,Kidney_10X_P4_6
46582,10X_P7_5,Kidney,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P7_5,Kidney_10X_P7_5


In [42]:
pd.options.display.max_colwidth = 1000

In [43]:
samples['output'] = output_prefix + samples['id'] + '.csv'
samples.head()

/home/ubuntu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,channel,tissue,ksizes,num_hashes,tenx,id,output
16639,10X_P4_7,Spleen,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P4_7,Spleen_10X_P4_7,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Spleen_10X_P4_7.csv"
20078,10X_P7_6,Spleen,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P7_6,Spleen_10X_P7_6,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Spleen_10X_P7_6.csv"
45065,10X_P4_5,Kidney,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P4_5,Kidney_10X_P4_5,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Kidney_10X_P4_5.csv"
45675,10X_P4_6,Kidney,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P4_6,Kidney_10X_P4_6,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Kidney_10X_P4_6.csv"
46582,10X_P7_5,Kidney,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P7_5,Kidney_10X_P7_5,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Kidney_10X_P7_5.csv"


In [45]:
samples_cleaned = samples.set_index("id")
samples_cleaned

,channel,tissue,ksizes,num_hashes,tenx,output
id,,,,,,
Spleen_10X_P4_7,10X_P4_7,Spleen,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P4_7,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Spleen_10X_P4_7.csv"
Spleen_10X_P7_6,10X_P7_6,Spleen,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P7_6,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Spleen_10X_P7_6.csv"
Kidney_10X_P4_5,10X_P4_5,Kidney,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P4_5,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Kidney_10X_P4_5.csv"
Kidney_10X_P4_6,10X_P4_6,Kidney,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P4_6,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Kidney_10X_P4_6.csv"
Kidney_10X_P7_5,10X_P7_5,Kidney,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P7_5,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Kidney_10X_P7_5.csv"


In [47]:
import json

folder = '../sourmash/maca/10x_spleen_kidney'

! mkdir -p $folder
assert samples_cleaned.index.is_unique
samples_cleaned.to_csv(f'{folder}/samples.csv')

config = 	{
		"program": "../../../reflow/sourmash_compute_10x.rf",
		"runs_file": "samples.csv"
	}

with open(f'{folder}/config.json', 'w') as f:
    json.dump(config, f)
    
! head -n 2 $folder/samples.csv $folder/config.json

==> ../sourmash/maca/10x_spleen_kidney/samples.csv <==
id,channel,tissue,ksizes,num_hashes,tenx,output
Spleen_10X_P4_7,10X_P4_7,Spleen,"21,27,33,51",1000,s3://czbiohub-maca/10x_data/10X_P4_7,"s3://olgabot-maca/10x/sourmash_compute/ksizes=21,27,33,51_num_hashes=1000/Spleen_10X_P4_7.csv"

==> ../sourmash/maca/10x_spleen_kidney/config.json <==
{"program": "../../../reflow/sourmash_compute_10x.rf", "runs_file": "samples.csv"}